<a href="https://colab.research.google.com/github/taaha3244/AI-Agents/blob/main/crewAI_agentic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 83.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from crewai import Agent, Task, Crew

In [13]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [14]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [15]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [16]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [17]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [18]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [19]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [20]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [21]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Rise of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries.
- Mention of the latest trends in AI technology and the key players leading the advancements.

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural Language Processing (NLP) and chatbots
- Computer vision and image recognition
- AI in healthcare, finance, and autonomous vehi

In [22]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has revolutionized various industries, shaping the way we interact with technology and paving the path for innovative solutions. The latest trends in AI technology and the key players leading the advancements have significantly impacted our daily lives. From machine learning algorithms to natural language processing, AI continues to push the boundaries of what is possible.

## Latest Trends in Artificial Intelligence
Machine learning and deep learning algorithms have become the cornerstone of AI innovation, allowing machines to learn from data and improve their performance over time. Natural Language Processing (NLP) and chatbots have transformed customer service interactions and personalized experiences. Additionally, computer vision and image recognition have enhanced security systems and automated processes. AI applications in healthcare, finance, and autonomous vehicles have revolutionized these industries, making them more efficient and accessible.

## Key Players in the AI Industry
Leading the charge in AI development are tech giants like Google, Amazon, Microsoft, and IBM, each with their own AI platforms and services. Google's Google Brain and TensorFlow have pushed the boundaries of AI research and development. Amazon's AWS AI services have made AI more accessible to businesses of all sizes. Microsoft's Azure AI and Cognitive Services have enabled companies to integrate AI into their existing systems. IBM's Watson AI has been at the forefront of AI-powered solutions for healthcare and enterprise applications. Not to be overlooked are startups like OpenAI and DeepMind, which continue to push the envelope in AI research and development.

## Noteworthy News in Artificial Intelligence
Recent developments in AI ethics and bias have raised concerns about the ethical implications of AI technology. Applications of AI in combating climate change have shown promising results in optimizing resource management and reducing environmental impact. The impact of AI on job automation and workforce transformation has sparked debates on the future of work and the need for upskilling and reskilling in a rapidly changing economy.

## Audience Analysis
Our target audience of business professionals, tech enthusiasts, and students interested in AI will find valuable insights in the latest technology trends and industry developments. Addressing their pain points regarding job displacement, ethical concerns, and complex AI concepts will provide a well-rounded perspective on the impact of AI on society.

## Call to Action
I encourage readers to stay informed about AI developments by subscribing to industry newsletters or attending AI conferences. Engaging in discussions about AI on social media platforms or online forums will foster a community of knowledge-sharing and collaboration.

By incorporating the latest trends and key players in the AI industry, this blog post aims to provide a comprehensive overview of the advancements in AI technology and its impact on various industries. Stay tuned for more updates on the ever-evolving field of Artificial Intelligence.